# wav2vec_test
- force align each annotation/wav snippet 
- save results in json file

In [6]:
import os
import json
import torch
import IPython
import torchaudio
import matplotlib.pyplot as plt
from dataclasses import dataclass


print(torch.__version__)
print(torchaudio.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.random.manual_seed(0)

2.2.2+cpu
2.2.2+cpu
cpu


In [7]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model().to(device)
labels = bundle.get_labels()

print(labels)

('-', '|', 'E', 'T', 'A', 'O', 'N', 'I', 'H', 'S', 'R', 'D', 'L', 'U', 'M', 'W', 'C', 'F', 'G', 'Y', 'P', 'B', 'V', 'K', "'", 'X', 'J', 'Q', 'Z')


In [8]:
import re

folder_path = 'C:\\Users\\barth\\gits\\pytorch_wav2vec\\test_data\\'
inputPath = folder_path + "input\\"
mediaPath = folder_path + "media_snippets\\"

def read_json_file(filename):
    try:
        with open(filename, 'r') as json_file:
            data = json.load(json_file)
            return data
    except Exception as e:
        print(f"Error reading JSON file: {e}")
        return None

def get_trellis(emission, tokens, blank_id=0):
    num_frame = emission.size(0)
    num_tokens = len(tokens)

    trellis = torch.zeros((num_frame, num_tokens))
    trellis[1:, 0] = torch.cumsum(emission[1:, blank_id], 0)
    trellis[0, 1:] = -float("inf")
    trellis[-num_tokens + 1 :, 0] = float("inf")

    for t in range(num_frame - 1):
        trellis[t + 1, 1:] = torch.maximum(
            # Score for staying at the same token
            trellis[t, 1:] + emission[t, blank_id],
            # Score for changing to the next token
            trellis[t, :-1] + emission[t, tokens[1:]],
        )
    return trellis

def plot():
    fig, ax = plt.subplots()
    img = ax.imshow(trellis.T, origin="lower")
    ax.annotate("- Inf", (trellis.size(1) / 5, trellis.size(1) / 1.5))
    ax.annotate("+ Inf", (trellis.size(0) - trellis.size(1) / 5, trellis.size(1) / 3))
    fig.colorbar(img, ax=ax, shrink=0.6, location="bottom")
    fig.tight_layout()

@dataclass
class Point:
    token_index: int
    time_index: int
    score: float

def backtrack(trellis, emission, tokens, blank_id=0):
    t, j = trellis.size(0) - 1, trellis.size(1) - 1

    path = [Point(j, t, emission[t, blank_id].exp().item())]
    while j > 0:
        # Should not happen but just in case
        assert t > 0

        # 1. Figure out if the current position was stay or change
        # Frame-wise score of stay vs change
        p_stay = emission[t - 1, blank_id]
        p_change = emission[t - 1, tokens[j]]

        # Context-aware score for stay vs change
        stayed = trellis[t - 1, j] + p_stay
        changed = trellis[t - 1, j - 1] + p_change

        # Update position
        t -= 1
        if changed > stayed:
            j -= 1

        # Store the path with frame-wise probability.
        prob = (p_change if changed > stayed else p_stay).exp().item()
        path.append(Point(j, t, prob))

    # Now j == 0, which means, it reached the SoS.
    # Fill up the rest for the sake of visualization
    while t > 0:
        prob = emission[t - 1, blank_id].exp().item()
        path.append(Point(j, t - 1, prob))
        t -= 1

    return path[::-1]

# Merge the labels
@dataclass
class Segment:
    label: str
    start: int
    end: int
    score: float

    def __repr__(self):
        return f"{self.label}\t({self.score:4.2f}): [{self.start:5d}, {self.end:5d})"

    @property
    def length(self):
        return self.end - self.start


def merge_repeats(path):
    i1, i2 = 0, 0
    segments = []
    while i1 < len(path):
        while i2 < len(path) and path[i1].token_index == path[i2].token_index:
            i2 += 1
        score = sum(path[k].score for k in range(i1, i2)) / (i2 - i1)
        segments.append(
            Segment(
                transcript[path[i1].token_index],
                path[i1].time_index,
                path[i2 - 1].time_index + 1,
                score,
            )
        )
        i1 = i2
    return segments

# Merge words
def merge_words(segments, separator="|"):
    words = []
    sounds = []
    i1, i2 = 0, 0
    while i1 < len(segments):
        
        if i2 >= len(segments) or segments[i2].label == separator:
            if i1 != i2:
                segs = segments[i1:i2]
                word = "".join([seg.label for seg in segs])
                score = sum(seg.score * seg.length for seg in segs) / sum(seg.length for seg in segs)
                words.append(Segment(word, segments[i1].start, segments[i2 - 1].end, score))
            i1 = i2 + 1
            i2 = i1
        else:
            i2 += 1
    print (words, sounds)
    return words, sounds

def segment_info(i):
    ratio = waveform.size(1) / trellis.size(0)
    word = word_segments[i]
    x0 = int(ratio * word.start)
    x1 = int(ratio * word.end)
    #print(f"{word.label} ({word.score:.2f}): {x0 / 44100:.3f} - {x1 / 44100:.3f} sec")
    return ([f"{word.label}", format(x0 / 44100,'.3f') , format(x1 / 44100, '.3f')])
    

for filename in os.listdir(mediaPath):
    counter  = 0
    
    if filename.endswith('.json'):
        jsonFile = mediaPath + filename
        json_data = read_json_file(jsonFile)
        for k, v in json_data.items():
            
            if v["text"] and len(v["text"]) > 0:
                
                text_clean = v["text"].upper().replace(' ', '|')
                transcript = text_clean
            
            else:
                transcript = "|"
            #print (text_clean)
            
            
            print (transcript)
            regex = re.compile('[^a-zA-Z|]')
            transcript = regex.sub('', transcript)
            
            
            print (transcript)
            
            dictionary = {c: i for i, c in enumerate(labels)}
            
            tokens = [dictionary[c] for c in transcript]
            #print(list(zip(transcript, tokens)))

            SPEECH_FILE = "C:/Users/barth/gits/pytorch_wav2vec/test_data/media_snippets/" + filename[:-5] + "\\" + k.replace(" ", "_") + ".wav" #Cathy_Samun_Wiliang_a1.wav"
            print (SPEECH_FILE)
            #bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
            
            #model = bundle.get_model().to(device)
            #labels = bundle.get_labels()
            
            
            with torch.inference_mode():
                waveform, _ = torchaudio.load(SPEECH_FILE)
                emissions, _ = model(waveform.to(device))
                emissions = torch.log_softmax(emissions, dim=-1)
            
            emission = emissions[0].cpu().detach()
            metadata = torchaudio.info(SPEECH_FILE)
            #print(metadata)
            
            trellis = get_trellis(emission, tokens)
           
            #plot()
            
            path = backtrack(trellis, emission, tokens)

            segments = merge_repeats(path)
            
            
            letter_list = []

            for segment in range(len(segments)):
                ratio = waveform.size(1) / trellis.size(0)

                x0 = int(ratio * segments[segment].start)
                x1 = int(ratio * segments[segment].end)

                #print(f"{word.label} ({word.score:.2f}): {x0 / 44100:.3f} - {x1 / 44100:.3f} sec")
                soundInfo = [f"{segments[segment].label}", format(x0 / 44100,'.3f') , format(x1 / 44100, '.3f')]
                soundInfo[1] = int(float(soundInfo[1]) * 1000) + v['timeStamp1']
                soundInfo[2] = int(float(soundInfo[2]) * 1000) + v['timeStamp1']

                # ingnore the word gaps
                if soundInfo[0] != "|": 
                    letter_list.append(soundInfo)
                
                
                
            word_segments, sounds = merge_words(segments)
            word_list = []
            for x in range(0, len(word_segments)):
                wordInfo = segment_info(x)
                wordInfo[1] = int(float(wordInfo[1]) * 1000) + v['timeStamp1']
                wordInfo[2] = int(float(wordInfo[2]) * 1000) + v['timeStamp1']
                print(wordInfo)
                
                word_list.append(wordInfo)


            print (word_list)
            print(v)
            v["tier_new"] = v["tiername"] + "_words"
            v["words"] = word_list
            v["letters"] = letter_list                 
            
            
            #print(transcript)
            #display_segment(-1)
            #IPython.display.Audio(SPEECH_FILE, rate=44100)

        print (json_data)
        with open(jsonFile, 'w') as json_file:
            json.dump(json_data, json_file, indent=2)

print (" +++ DONE +++ " )

WELL|BEFORE|MY|HIGH|SCHOOL|YEAH|
WELL|BEFORE|MY|HIGH|SCHOOL|YEAH|
C:/Users/barth/gits/pytorch_wav2vec/test_data/media_snippets/AusESL_MF_002_Stella_anon\LaBB-CAT_PNT__Stella__a329.wav
[WELL	(0.89): [    0,    58), BEFORE	(0.88): [   68,   169), MY	(0.49): [  170,   172), HIGH	(0.91): [  250,   300), SCHOOL	(0.75): [  305,   341), YEAH	(0.87): [  361,   426)] []
['WELL', 8460, 8882]
['BEFORE', 8954, 9688]
['MY', 9696, 9710]
['HIGH', 10277, 10641]
['SCHOOL', 10677, 10939]
['YEAH', 11084, 11556]
[['WELL', 8460, 8882], ['BEFORE', 8954, 9688], ['MY', 9696, 9710], ['HIGH', 10277, 10641], ['SCHOOL', 10677, 10939], ['YEAH', 11084, 11556]]
{'tiername': 'LaBB-CAT_PNT_ Stella ', 'annoID': 'a329', 'timeSlotRef1': 'ts6', 'timeSlotRef2': 'ts7', 'timeStamp1': 8460, 'timeStamp2': 11680, 'text': 'Well before my high school yeah ', 'words': [], 'phonemes': []}
I|STUDY|THE|THERE|A|COUPLE|OF|YEARS|IN|OUR|HOMETOWN|
I|STUDY|THE|THERE|A|COUPLE|OF|YEARS|IN|OUR|HOMETOWN|
C:/Users/barth/gits/pytorch_wav2vec/tes

[YEP	(0.86): [    0,    67)] []
['YEP', 41600, 42090]
[['YEP', 41600, 42090]]
{'tiername': 'LaBB-CAT_PNT_ Stella ', 'annoID': 'a1056', 'timeSlotRef1': 'ts43', 'timeSlotRef2': 'ts46', 'timeStamp1': 41600, 'timeStamp2': 42090, 'text': '!yep', 'words': [], 'phonemes': []}
THREE|YEARS|FOR|MAST~|UH|FOR|BACHELOR|DEGREE|AND|THEN|TWO|YEARS|FOR|MASTER|DEGREE
THREE|YEARS|FOR|MAST|UH|FOR|BACHELOR|DEGREE|AND|THEN|TWO|YEARS|FOR|MASTER|DEGREE
C:/Users/barth/gits/pytorch_wav2vec/test_data/media_snippets/AusESL_MF_002_Stella_anon\LaBB-CAT_PNT__Stella__a331.wav
[THREE	(0.86): [    0,    46), YEARS	(0.84): [   51,    87), FOR	(0.81): [   98,   131), MAST	(0.89): [  134,   168), UH	(0.64): [  203,   230), FOR	(0.84): [  251,   282), BACHELOR	(0.81): [  293,   340), DEGREE	(0.85): [  375,   437), AND	(0.79): [  456,   467), THEN	(0.81): [  479,   498), TWO	(0.88): [  520,   565), YEARS	(0.84): [  572,   611), FOR	(0.85): [  625,   649), MASTER	(0.88): [  658,   704), DEGREE	(0.89): [  730,   794)] []
['TH

[PEOPLE	(0.79): [    0,    36), GET	(0.81): [  113,   135), SOLVE	(0.79): [  157,   188), PROBLEM	(0.83): [  213,   286), LIKE	(0.71): [  288,   306), TOGETHER	(0.81): [  316,   387)] []
['PEOPLE', 88845, 89107]
['GET', 89667, 89827]
['SOLVE', 89988, 90213]
['PROBLEM', 90395, 90926]
['LIKE', 90941, 91072]
['TOGETHER', 91145, 91661]
[['PEOPLE', 88845, 89107], ['GET', 89667, 89827], ['SOLVE', 89988, 90213], ['PROBLEM', 90395, 90926], ['LIKE', 90941, 91072], ['TOGETHER', 91145, 91661]]
{'tiername': 'LaBB-CAT_PNT_ Stella ', 'annoID': 'a1068', 'timeSlotRef1': 'ts93', 'timeSlotRef2': 'ts94', 'timeStamp1': 88845, 'timeStamp2': 91785, 'text': 'people get solve problem like together ', 'words': [], 'phonemes': []}
YEAH|AND|VERY|HARD|TO|TO|TO|DO|SOMETHING|BY|YOURSELF|AND
YEAH|AND|VERY|HARD|TO|TO|TO|DO|SOMETHING|BY|YOURSELF|AND
C:/Users/barth/gits/pytorch_wav2vec/test_data/media_snippets/AusESL_MF_002_Stella_anon\LaBB-CAT_PNT__Stella__a1069.wav
[YEAH	(0.88): [    0,    49), AND	(0.46): [  127,   

[YEAH	(0.90): [    0,    87)] []
['YEAH', 116088, 116725]
[['YEAH', 116088, 116725]]
{'tiername': 'LaBB-CAT_PNT_ Stella ', 'annoID': 'a1561', 'timeSlotRef1': 'ts117', 'timeSlotRef2': 'ts118', 'timeStamp1': 116088, 'timeStamp2': 116725, 'text': '!yeah', 'words': [], 'phonemes': []}
!YEAH|!YEAH|YEAH
YEAH|YEAH|YEAH
C:/Users/barth/gits/pytorch_wav2vec/test_data/media_snippets/AusESL_MF_002_Stella_anon\LaBB-CAT_PNT__Stella__a334.wav
[YEAH	(0.82): [    0,    61), YEAH	(0.85): [   65,   101), YEAH	(0.85): [  129,   210)] []
['YEAH', 118210, 118654]
['YEAH', 118684, 118946]
['YEAH', 119150, 119740]
[['YEAH', 118210, 118654], ['YEAH', 118684, 118946], ['YEAH', 119150, 119740]]
{'tiername': 'LaBB-CAT_PNT_ Stella ', 'annoID': 'a334', 'timeSlotRef1': 'ts121', 'timeSlotRef2': 'ts125', 'timeStamp1': 118210, 'timeStamp2': 119740, 'text': '!Yeah !yeah yeah', 'words': [], 'phonemes': []}
OKAY
OKAY
C:/Users/barth/gits/pytorch_wav2vec/test_data/media_snippets/AusESL_MF_002_Stella_anon\LaBB-CAT_PNT__Stell

[AND	(0.90): [    0,    37), THEN	(0.83): [   59,    82), AFTER	(0.86): [  100,   183), THEN	(0.71): [  225,   240), I	(0.91): [  264,   268), GOT	(0.75): [  292,   313), A	(0.97): [  314,   315), GOT	(0.83): [  389,   431), A	(0.72): [  523,   528), BABY	(0.90): [  537,   565)] []
['AND', 155835, 156104]
['THEN', 156264, 156431]
['AFTER', 156561, 157164]
['THEN', 157470, 157579]
['I', 157753, 157782]
['GOT', 157956, 158109]
['A', 158116, 158123]
['GOT', 158661, 158966]
['A', 159635, 159671]
['BABY', 159736, 159940]
[['AND', 155835, 156104], ['THEN', 156264, 156431], ['AFTER', 156561, 157164], ['THEN', 157470, 157579], ['I', 157753, 157782], ['GOT', 157956, 158109], ['A', 158116, 158123], ['GOT', 158661, 158966], ['A', 159635, 159671], ['BABY', 159736, 159940]]
{'tiername': 'LaBB-CAT_PNT_ Stella ', 'annoID': 'a1084', 'timeSlotRef1': 'ts157', 'timeSlotRef2': 'ts158', 'timeStamp1': 155835, 'timeStamp2': 160085, 'text': 'and then after then I got a got a baby ', 'words': [], 'phonemes': [

...

[YEAH	(0.93): [    0,    74)] []
['YEAH', 1518110, 1518655]
[['YEAH', 1518110, 1518655]]
{'tiername': 'INT_ Sylvie', 'annoID': 'a1475', 'timeSlotRef1': 'ts1784', 'timeSlotRef2': 'ts1787', 'timeStamp1': 1518110, 'timeStamp2': 1518655, 'text': 'yeah', 'words': [], 'phonemes': []}
HAVE|YOU|EVER|BEEN|OFFENDED|BY|PEOPLE'S|TELLING|UH|LIKE|JOKES|YOU|DON'T|FIND|APPROPRIATE
HAVE|YOU|EVER|BEEN|OFFENDED|BY|PEOPLES|TELLING|UH|LIKE|JOKES|YOU|DONT|FIND|APPROPRIATE
C:/Users/barth/gits/pytorch_wav2vec/test_data/media_snippets/AusESL_MF_002_Stella_anon\INT__Sylvie_a908.wav
[HAVE	(0.80): [    0,    19), YOU	(0.83): [   24,    44), EVER	(0.79): [   55,    82), BEEN	(0.76): [   89,   106), OFFENDED	(0.84): [  119,   197), BY	(0.85): [  225,   267), PEOPLES	(0.73): [  305,   340), TELLING	(0.85): [  360,   417), UH	(0.86): [  530,   539), LIKE	(0.69): [  552,   577), JOKES	(0.84): [  674,   732), YOU	(0.83): [  733,   766), DONT	(0.87): [  771,   796), FIND	(0.68): [  811,   838), APPROPRIATE	(0.88): [  84

[YES	(0.91): [    0,    87)] []
['YES', 1593361, 1593998]
[['YES', 1593361, 1593998]]
{'tiername': 'INT_ Sylvie', 'annoID': 'a910', 'timeSlotRef1': 'ts1847', 'timeSlotRef2': 'ts1849', 'timeStamp1': 1593361, 'timeStamp2': 1593998, 'text': 'Yes', 'words': [], 'phonemes': []}
YEAH|DO|THEY|VISIT|HIM|VERY|OFTEN
YEAH|DO|THEY|VISIT|HIM|VERY|OFTEN
C:/Users/barth/gits/pytorch_wav2vec/test_data/media_snippets/AusESL_MF_002_Stella_anon\INT__Sylvie_a911.wav
[YEAH	(0.96): [    0,   162), DO	(0.80): [  173,   186), THEY	(0.05): [  187,   191), VISIT	(0.58): [  192,   203), HIM	(0.89): [  213,   238), VERY	(0.66): [  273,   285), OFTEN	(0.87): [  296,   370)] []
['YEAH', 1621016, 1622194]
['DO', 1622274, 1622369]
['THEY', 1622376, 1622405]
['VISIT', 1622412, 1622492]
['HIM', 1622565, 1622747]
['VERY', 1623002, 1623089]
['OFTEN', 1623169, 1623707]
[['YEAH', 1621016, 1622194], ['DO', 1622274, 1622369], ['THEY', 1622376, 1622405], ['VISIT', 1622412, 1622492], ['HIM', 1622565, 1622747], ['VERY', 1623002,

[TRAFFIC	(0.88): [    0,    79)] []
['TRAFFIC', 1671028, 1671610]
[['TRAFFIC', 1671028, 1671610]]
{'tiername': 'INT_ Sylvie', 'annoID': 'a1523', 'timeSlotRef1': 'ts1939', 'timeSlotRef2': 'ts1941', 'timeStamp1': 1671028, 'timeStamp2': 1671610, 'text': 'traffic', 'words': [], 'phonemes': []}
YES
YES
C:/Users/barth/gits/pytorch_wav2vec/test_data/media_snippets/AusESL_MF_002_Stella_anon\INT__Sylvie_a1529.wav
[YES	(0.92): [    0,    62)] []
['YES', 1691781, 1692236]
[['YES', 1691781, 1692236]]
{'tiername': 'INT_ Sylvie', 'annoID': 'a1529', 'timeSlotRef1': 'ts1949', 'timeSlotRef2': 'ts1951', 'timeStamp1': 1691781, 'timeStamp2': 1692236, 'text': 'yes', 'words': [], 'phonemes': []}
YEAH
YEAH
C:/Users/barth/gits/pytorch_wav2vec/test_data/media_snippets/AusESL_MF_002_Stella_anon\INT__Sylvie_a1530.wav
[YEAH	(0.94): [    0,   163)] []
['YEAH', 1696650, 1697841]
[['YEAH', 1696650, 1697841]]
{'tiername': 'INT_ Sylvie', 'annoID': 'a1530', 'timeSlotRef1': 'ts1958', 'timeSlotRef2': 'ts1960', 'timeStamp

[LIKE	(0.90): [    0,    37), Y	(0.71): [   39,    45), YOU	(0.56): [  105,   112), YOU	(0.87): [  116,   140), CANT	(0.74): [  153,   168), REALLY	(0.87): [  169,   217)] []
['LIKE', 1758418, 1758688]
['Y', 1758702, 1758746]
['YOU', 1759183, 1759235]
['YOU', 1759264, 1759438]
['CANT', 1759533, 1759643]
['REALLY', 1759650, 1760000]
[['LIKE', 1758418, 1758688], ['Y', 1758702, 1758746], ['YOU', 1759183, 1759235], ['YOU', 1759264, 1759438], ['CANT', 1759533, 1759643], ['REALLY', 1759650, 1760000]]
{'tiername': 'INT_ Sylvie', 'annoID': 'a1544', 'timeSlotRef1': 'ts2015', 'timeSlotRef2': 'ts2018', 'timeStamp1': 1758418, 'timeStamp2': 1760000, 'text': "like y~ you you can't really", 'words': [], 'phonemes': []}
GET|ACCUSTOMED|TO|UH
GET|ACCUSTOMED|TO|UH
C:/Users/barth/gits/pytorch_wav2vec/test_data/media_snippets/AusESL_MF_002_Stella_anon\INT__Sylvie_a1545.wav
[GET	(0.84): [    0,    31), ACCUSTOMED	(0.85): [   81,   156), TO	(0.94): [  189,   239), UH	(0.93): [  268,   302)] []
['GET', 176069

 +++ DONE +++ 


In [18]:
def display_segment(i):
    ratio = waveform.size(1) / trellis.size(0)
    word = word_segments[i]
    x0 = int(ratio * word.start)
    x1 = int(ratio * word.end)
    print(f"{word.label} ({word.score:.2f}): {x0 / 44100:.3f} - {x1 / 44100:.3f} sec")
    segment = waveform[:, x0:x1]

    return IPython.display.Audio(segment.numpy(), rate=44100)

display_segment(1)

TAMAT (0.78): 0.618 - 0.851 sec


In [8]:
IPython.display.Audio(SPEECH_FILE, rate=44100)

In [ ]:
"C:\Users\barth\gits\pytorch_wav2vec\test_data\media_snippets\Cathy_Samun_Wiliang_a1.wav"
"C:\Users\barth\gits\pytorch_wav2vec\test_data\media_snippets\Cathy Samun Wiliang_a1.wav"

"C:\Users\barth\gits\pytorch_wav2vec\test_data\media_snippets\Cathy_Samun_Wiliang_a53.wav"